# 零噪声外插抑噪方法

*版权所有 (c) 2022 百度量子计算研究所，保留所有权利。*

## 内容概要

本教程介绍量子计算中一种常用的噪声处理方法：零噪声外插抑噪法（Zero-Noise Extrapolation，ZNE）。我们首先会简明扼要地阐述这一方法的原理依据和实现方式，然后使用 Clifford 序列为例来演示如何基于 QEP 调用该方法。

## 理论

在量子噪声缓释（Quantum Error Mitigation，QEM）领域，零噪声外插抑噪法是一种消除量子门噪声强有力的技术。事实上，ZNE 方法并不直接抑制量子计算过程中的量子门的固有噪声，而是能够帮助我们获得等效的噪声缓释之后的计算结果。其主要依据的思想是，我们虽然不知道噪声的具体形式和如何控制噪声源，但如果能够增大或等效地增大错误率，便能够利用不同错误率下的计算结果，外推至错误率为 $0$ 时的计算结果，即理想的计算结果。这也是 ZNE 方法的主要优势所在。这一方法尤其适用于形如 $\textrm{Tr}[A\rho]$ 的期望值计算任务。在多数情况下，ZNE 方法都能够表现出良好的效果，是量子噪声缓释领域一种相当具有代表性的方法。

ZNE 方法的实施流程如下图所示。由图可知，ZNE 方法的实施需要两步：噪声缩放和错误外推。在众多噪声缩放技术中，电路层面的 Unitary Folding 方法是一种兼具较高抽象程度和可行性的方法。在该方法框架下，需要接连插入含噪声量子门并保证整体电路模型在理想情况下效果与原电路相同，但量子噪声的作用效果会明显增强，借此能够获得等效的噪声缩放了情况下的量子终态。在外插估计阶段，常有的算法法有“多项式外插”和“指数外插”等。即便没有通用的数学公式能够严格量化这种噪声缩放技术和外插算法能够消除的量子计算结果的误差，但原理上和经验上 ZNE 方法所得到的噪声缓释后的数值结果相比原本的含噪声期望值往往有较大的提升。

![zne-profile](figures/zne-profile.png "图1：ZNE 方法流程。")

### 电路噪声缩放

电路噪声缩放方法被称为酉折叠（Digital Unitary Folding）。由于该方法是在量子门或量子电路实现的，抽象层度更高，因此称之为 Digital。从实际的实现手段和效果上来看，又能称之为 Identity Insertion —— 通过在原有量子电路的中施加多个 $U^\dagger U$ 形式的等效单位算子（identity-equivalent operator）来扩展电路深度，以期达到等效地放大噪声的效果。显然，这种方法假设潜在的噪声效应随着量子程序的深度和（或门的数目）增大时而增大，并且如能根据对噪声模型的某些假设，这种方法下噪声放大的定量效果一般能够在解析层面推导出，而后便能够采用针对性的外插方法估计出噪声系数为 $0$ 时的量子计算结果。


以下我们使用 $U$ 代表整个量子电路对应的幺正算符，$L_i$ 代表了量子电路的第 $i$ 层（quantum layer） 或者第 $i$ 个量子门（quantum gate）。借助这种表示，

$$
U = L_d L_{d-1}\cdots L_1,
$$

可以将 Identity Insertion 方法进一步分为量子电路层面（circuit-level）和量子门层面的折叠方式。

#### 量子电路折叠（circuit-level folding）

量子电路层面的折叠方式即根据

$$
U \to U(U^\dagger U)^n,
$$

我们称 $n$ 为折叠系数，它使得量子电路的深度扩展为原来的 $2n+1$ 倍。若要实现更“精细”的缩放倍数，可以另外在量子门层面多做几次折叠，算符表示为

$$
U \to U(U^\dagger U)^n L_{i_1}^\dagger L_{i_2}^\dagger \cdots L_{i_s}^\dagger L_{i_s}\cdots L_{i_2} L_{i_1},
$$

其中 $1 \leq i_j \leq d$，这使得整个线路深度扩展为原来的

$$
\lambda  = \frac{(2n+1)d + 2s}{d} = \frac{2(nd+1)}{d} +1 := \frac{2k}{d} + 1.
$$

由此便得出我们的缩放策略：

1. 针对深度为 $d$ 的量子电路，定义想要获得的缩放系数 $\lambda$；

2. 计算 
    $$k = \left\lceil\frac{d(\lambda - 1)}{2}\right\rceil,\quad\frac{k}{d}=n \;\text{mod}\; s,$$
    得出更精细的缩放参数 $n$、$s$，其中 $\lceil\cdot\rceil$ 表示上取整运算；
    
3. 根据缩放参数得到折叠后量子电路对应的算符序列。

步骤 3 中在基于“精细”缩放参数 $s$ 得到额外折叠一次的量子门的指标集合。


#### 量子门层面折叠（layer-level folding）

以上电路层面的折叠所针对的主体等效单位算子是 $U^\dagger U$，而由于 $U=L_d\cdots L_1$，若将 $L_i^\dagger L_i$ 作为主体 identity 算子，则能够定义出量子门层面的折叠方法。

![layer-folding-profile](figures/zne-layer-folding.png "图2：量子门层面折叠。")

同样地，若需要实现更“精细”的折叠，不同量子门的扩展倍数也需要有差别。如以上示意图中 $i\in S$ 而 $j\notin S$，需要对指标集

$$
S = \left\{ i_1,i_2,\cdots,i_s \right\} \subseteq \left\{1,2,\cdots,d \right\}
$$

对应的量子门多做一次折叠。

需要指出的是，实施该方法的过程中，更多的是首先对电路分层，得到的每一个电路层（layer）就有可能包含一个或多个量子门，因此我们习惯称之为量子门层面折叠（ layer-level folding）而非量子门折叠）（gate-level folding）；但出于方便，在我们的叙述中假定一个“层”仅包含单个“量子门”，则此处量子门与量子电路层在表述上同义。

无论在电路层面还是量子门层面做折叠，都需要得到控制“精细”缩放程度的一组量子门下标的指标集合 $S= \left\{ i_1, i_2,\cdots,i_s \right\}$。目前有左向、右向和随机选取三种较为普遍的三种方法。例如，对于深度为 $d$ 的电路选取大小为 $s$ （$1\leq s\leq d$）的指标集时，采用不同方式会得到以下结果：

- 左向（from left）：$S = \left\{1,\cdots ,s\right\}$；

- 右向（from right）：$S = \left\{d,\cdots ,d-s+1\right\}$；

- 随机（at random）：$i_j$ 随机选取于 $\left\{1,\cdots, d \right\}$ 。

**电路噪声缩放示例**

更直观的，我们采用 QCompute 和 QEP 框架构建一个四比特 GHZ 态

$$
|\psi\rangle= \frac{|0000\rangle +|1111\rangle}{\sqrt{2}}
$$

的制备电路，分别在量子门层面和量子电路层面并采用不同的“精细”指标集选取方式做折叠来演示。

In [4]:
from QCompute import *
from qcompute_qep.utils.circuit import print_circuit
from qcompute_qep.mitigation import zne


qp = QEnv()
qreg = qp.Q.createList(4)
H(qreg[0])
for i in range(1, 4):
    CX(qreg[0], qreg[i])
# print circuit in text style
print_circuit(qp.circuit)

0: ---H---@---@---@---
          |   |   |   
1: -------X-----------
              |   |   
2: -----------X-------
                  |   
3: ---------------X---


'0: ---H---@---@---@---\n          |   |   |   \n1: -------X-----------\n              |   |   \n2: -----------X-------\n                  |   \n3: ---------------X---'

该电路的深度为 $d=4$，若设定缩放系数 $\lambda=6$，按照如上算法步骤计算出 folding 参数中 $n=2$，$s=2$，对应到算符表示为

$$
U = L_4 L_3 L_2 L_1 = CNOT_{03}*CNOT_{02}*CNOT_{01}*H_{0} \to U(U^\dagger U)L_a^\dagger L_b^\dagger L_b L_a.
$$


*电路层面（circuit-level）folding*

1. 左向选取

In [25]:
folder = zne.CircuitFolder() # construct a circuit-level folder

In [26]:
qp_folded = folder(qp, scale_factor=lam, method='left')
print_circuit(qp_folded.circuit)

0: ---H---@---@---@---=---@---@---@---H---=---H---@---@---@---=---@---@---@---H---=---H---@---@---@---=---@---H---H---@---
          |   |   |   =   |   |   |       =       |   |   |   =   |   |   |       =       |   |   |   =   |           |   
1: -------X-----------=-----------X-------=-------X-----------=-----------X-------=-------X-----------=---X-----------X---
              |   |   =   |   |           =           |   |   =   |   |           =           |   |   =                   
2: -----------X-------=-------X-----------=-----------X-------=-------X-----------=-----------X-------=-------------------
                  |   =   |               =               |   =   |               =               |   =                   
3: ---------------X---=---X---------------=---------------X---=---X---------------=---------------X---=-------------------


2. 右向选取

In [24]:
qp_folded = folder(qp, lam, method='right')
print_circuit(qp_folded.circuit)

0: ---H---H---H---H---H---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---
                          |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   
1: -----------------------X---X---X---X---X-----------------------------------------------------------
                                              |   |   |   |   |   |   |   |   |   |   |   |   |   |   
2: -------------------------------------------X---X---X---X---X---X---X-------------------------------
                                                                          |   |   |   |   |   |   |   
3: -----------------------------------------------------------------------X---X---X---X---X---X---X---


3. 随机选取

In [23]:
qp_folded = folder(qp, scale_factor=lam, method='random')
print_circuit(qp_folded.circuit)

0: ---H---H---H---H---H---H---H---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---
                                  |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   
1: -------------------------------X---X---X---X---X---------------------------------------------------
                                                      |   |   |   |   |   |   |   |   |   |   |   |   
2: ---------------------------------------------------X---X---X---X---X---X---X-----------------------
                                                                                  |   |   |   |   |   
3: -------------------------------------------------------------------------------X---X---X---X---X---


*量子门层面（layer-level）folding*

1. 左向选取

In [27]:
folder = zne.LayerFolder() # construct a layer-level folder

In [28]:
qp_folded = folder(qp, lam, method='left')
print_circuit(qp_folded.circuit)

0: ---H---H---H---H---H---H---H---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---
                                  |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   
1: -------------------------------X---X---X---X---X---X---X-------------------------------------------
                                                              |   |   |   |   |   |   |   |   |   |   
2: -----------------------------------------------------------X---X---X---X---X-----------------------
                                                                                  |   |   |   |   |   
3: -------------------------------------------------------------------------------X---X---X---X---X---


2. 右向选取

In [30]:
qp_folded = folder(qp, lam, method='right')
print_circuit(qp_folded.circuit)

0: ---H---H---H---H---H---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---
                          |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   
1: -----------------------X---X---X---X---X-----------------------------------------------------------
                                              |   |   |   |   |   |   |   |   |   |   |   |   |   |   
2: -------------------------------------------X---X---X---X---X---X---X-------------------------------
                                                                          |   |   |   |   |   |   |   
3: -----------------------------------------------------------------------X---X---X---X---X---X---X---


3. 随机选取

In [36]:
qp_folded = folder(qp, lam, method='random')
print_circuit(qp_folded.circuit)

0: ---H---H---H---H---H---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---@---
                          |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   
1: -----------------------X---X---X---X---X-----------------------------------------------------------
                                              |   |   |   |   |   |   |   |   |   |   |   |   |   |   
2: -------------------------------------------X---X---X---X---X---X---X-------------------------------
                                                                          |   |   |   |   |   |   |   
3: -----------------------------------------------------------------------X---X---X---X---X---X---X---


尽管在理论上，由于 $L^\dagger L$ 作为等效单位算子与其他任何算子都是对易的，量子门层面折叠方法实质上与量子电路层面折叠的差别并不大，都可以统一表示为

$$
U \to U(U^\dagger U)^n L_{i_1}^\dagger L_{i_2}^\dagger \cdots L_{i_s}^\dagger L_{i_s}\cdots L_{i_2} L_{i_1},
$$

但在实际的含噪声量子电路中其造成的噪声缩放效果不尽相同。


### 含噪期望值外插

通常我们所关心的量子计算结果 $E$ 可以表示为噪声系数/强度的函数， $E=E(\lambda)$。当我们通过 unitary folding 手段获得了一组不同噪声情况下的期望值，就能够根据先验假定地关于 $E(\lambda)$ 和  $\lambda$ 之间关系做拟合和外推，即根据已有数据点 $\left\{E(\lambda_j)\right\}_{j=0}^n$ 来估计零噪声下的期望值 $E(0)$。通常情况下，在假定了 $E(\lambda)$ 的形式或根据物理规律得到合理的形式表述后，估计 $E(0)$ 的任务就是一个回归问题 —— 首先通过已有数据点 $\left\{E(\lambda_j)\right\}_{j=0}^n$ 做数值拟合得到 $\lambda$ 和 $E(\lambda)$ 之间的函数形式，而后外推至零噪声下的期望值 $E(0)$，同时也能够定量分析外插估计的效果。$E(\lambda)$ 关于 $\lambda$ 的比较普遍的表示形式有多项式（Polynomial）表示和指数多项式（Poly-exponential）表示等，相应的外插法也称作多项式外插（Polynomial Extrapolation）和指数多项式（Poly-exponential Extrapolation）外插。

![extrapolation-profile](figures/zne-extrapolation.png "图3：零噪声外插法。")


#### 多项式外插

当假定了 $E(\lambda)$ 是 $\lambda$ 的多项式形式时，这一估计问题称作“多项式插值”，

$$
E(\lambda)=\sum_{k=0}^d c_k\lambda^k\Rightarrow E(0)=c_0.
$$

多项式回归实则是基于多项式基函数构造的多元线性回归 — 使用多项式基函数 $\varphi\in \mathbb{R}^{d_\varphi}$ 变换样本数据到更高维的表达 $(1,\lambda)\stackrel{\varphi}{\to} (1,\lambda,\cdots,\lambda^d)$。而线性回归模型要求当数据点数（$n+1$）不小于 $d$ 阶多项式未知参数个数（$d+1$）时，其效果才能够有较好的保证。

至于回归的具体步骤，可根据该多项式回归模型以及已经得到的得到 $n+1$ 个数据 $\left\{\lambda_j,E(\lambda_j)\right\}_{j=0}^n$， 将对应公式的线性方程组形式，

$$
\begin{pmatrix}
E_0\\
E_1\\
\vdots\\
E_n
\end{pmatrix}
=
\begin{pmatrix}
1 & \lambda_0 & \lambda_0^2 & \cdots & \lambda_0^d\\
1 & \lambda_1 & \lambda_1^2 & \cdots & \lambda_1^d\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
1 & \lambda_n & \lambda_n^2 & \cdots & \lambda_n^d\\
\end{pmatrix}
\begin{pmatrix}
1\\
c_1\\
\vdots\\
c_d
\end{pmatrix}
+
\begin{pmatrix}
\epsilon_0\\
\epsilon_1\\
\vdots\\
\epsilon_n
\end{pmatrix}
$$

或当使用纯矩阵表示法时，将其写为

$$
\vec{E}=\mathbf{\Lambda}\vec{c}+\vec{\epsilon},
$$

其中 $\Lambda\in \mathbb{R}^{(n+1)\times (d+1)}$ 称作样本矩阵，$\vec{\epsilon}$ 为估计误差。根据最小二乘法确定回归系数的矩阵计算公式，

$$
\hat{\vec{c}}=(\Lambda^T\Lambda)^{-1}\Lambda^T\vec{E},
$$

其中 $(\Lambda^T\Lambda)^{-1}$ 是方阵 $\Lambda^T\Lambda$ 的伪逆运算，而向量 $\hat{\vec{c}}$ 的第一个元素 $\hat{c}_0$ 即是“零噪声”情况下的插值估计结果。

例如，下图给出了根据三个数据点做二阶多项式拟合、然后外推至 $E(0)$ 的结果的示意图，外插后的数据相当接近理想标定值 $1$。

![poly-extrapolation](figures/zne-poly-extrapolation.png "图4：Richardson 外插。")


上述这种“拟合多项式函数阶数较数据点数少一”的情况也被称为 Richardson 外插，是一种高效且常用的外插方法之一。然而经验表明，当缩放系数差距较大、并且系数个数不太多时候 Richardson 外插的效果表现会更好，否则不然。当使用一阶多项式拟合时，就简化得到线性外插的情形，这种情形下关系假定虽然简单、拟合程度不能说是最优，但效果更加稳健。

#### 指数外插

$E(\lambda)$ 为指数多项式形式时，

$$
E(\lambda)=a\pm e^{z(\lambda)}=a\pm e^{\sum_{k=0}^d z_k\lambda^k}\Rightarrow E(0)=a\pm e^{z_0}
$$

包含 $d+2$ 个参数，并且从物理上考虑，该指数多项式需要满足

$$
\lim_{\lambda\to\infty}E(\lambda)=a\Leftrightarrow \lim_{\lambda\to\infty}z(\lambda)=-\infty
$$

在这种形式下的插值做法会考虑参数 $a$ 已知和未知两种情况：

1. 当根据物理规律能够从原理上确定当 $\lambda\to\infty$ 时参数 $a$ 的值时（例如当噪声速率趋于 $\infty$ 时系统处于最大混态，容易根据统计力学的结论得到相应力学量期望值），所需要做的工作只是 $d+1$ 个参数时的多项式插值，此时考虑以下步骤：
   - 对因变量数据变换，$E_j \to y_j^{'}:=\log(|E_j-a|+\epsilon)$，其中 $\epsilon$ 表示一大于 0 的正则化常数；
   - 根据 $\left\{\lambda_j,y_j^{'}\right\}$ 对 $z(\lambda)$ 做多项式拟合；
   - 外推至 $z(\lambda=0)=z_0$ 的值；
   - 得到结果 $E(0)=a\pm e^{z_0}$。
2. $a$ 未知，需要做非线性拟合。尽管通过数值计算也便于实现，但外插值的精度难以很好地得到保证。

指数型插值是多项式指数中阶数 $d=1$ 的特例，

$$
E(\lambda)=a\pm \exp{(z_0+z_1\lambda)}=a_0+be^{-c\lambda}.
$$

在酉折叠的零噪声外插抑噪方法中，针对全局效果的退极化信道噪声模型会得到这种表示。

例如，下图给出了根据二十个数据点做指数多项式拟合、然后外推至 $E(0)$ 的结果的示意图，外插后的数据相当接近理想标定值 $2$。

![exp-extrapolation](figures/zne-exp-extrapolation.png "图5：指数外插。")


## 实践

### 计算任务描述

我们将分别基于单、双单比特的 Clifford 电路做随机基准测试（random benchmarking），验证 ZNE 方法的可行性。

Clifford 序列包含了一组连续的随机 Clifford 量子门，常常被用作量子电路性能的基准测试。为了便于定标理想的力学量期望值，我们考虑包含了长度为 $n$ 的 Clifford 随机序列和一个相应逆运算的恒等（identity-equivalent）量子电路。如下图所示，$C_j$ 指代随机生成的 Clifford 幺正门，$C_{\rm inv}$ 指代前 $n$ 个 Clifford 量子门连续作用效果的等效逆门，使得

$$
C_{\rm inv}C_n C_{n-1}\cdots C_1=I.
$$


#### 单比特 Clifford 电路计算任务


我们将测试 ZNE 方法在初态为 $|0\rangle =\begin{pmatrix} 1\\0\end{pmatrix}$、测量算符为 $|0\rangle$ 态投影子 $A=\begin{pmatrix} 1&0\\0&0\end{pmatrix}$、长度为 $n+1$ 的恒等量子电路条件下的表现效果。


![single-clifford-circuit](figures/zne-single-clifford.png "图6：单量子比特恒等量子电路。")


理想情况下，量子态经过任意深度的恒等量子电路的作用，所得到的终态仍然会是 $|0\rangle$，测量算子的理想期望值也总保持为 $\langle A\rangle_{\rm ideal}=1$。

由于量子噪声的存在，并且施加更多的量子门会引入更大的噪声效应，恒等量子电路越深，量子系统终态会愈加偏离 $|0\rangle$，期望值 $\langle A\rangle_{\rm noisy}$ 的结果将会越偏离 $1$。


#### 两比特 Clifford 电路计算任务

更复杂的，我们将考虑将两比特 Clifford 电路加到贝尔态

$$
|\phi^+\rangle=\frac{|00\rangle + |11\rangle}{\sqrt{2}}
$$

的制备电路后面做随机测试，最终测试算子 $Z_1 Z_2$ 的期望。理想情况下该期望值 $\langle \phi^+ |Z_1Z_2 | \phi^+ \rangle = 1$。

![two-clifford-circuit](figures/zne-two-clifford.png "图7：双量子比特恒等量子电路。")

### 含噪和修正结果

在计算任务确认之后，给出“量易伏+真机”的具体实现方案。直接上代码片段。注意：代码已经按照约定划分为 QProgram, QComputer, calculator 三大块，方便后面的 ZNE 演示。

**导入必要的模块和函数**

在您成功安装最新版的量噪后，您可以按照本教程运行以下程序。为了顺利运行本教程，您需要从量噪和相关的 Python 库中带入以下模块或和函数：

In [46]:
import copy
from QCompute import *
from QCompute.QPlatform.QOperation import RotationGate
from QCompute.QPlatform.QOperation import QOperation
import numpy as np
import functools
from utils import clifford
import json
from mitigation.utils import visualization
from mitigation import zne
import matplotlib.pyplot as plt
from typing import Optional, List
import utils
from pprint import pprint

from utils.circuit import print_circuit
from utils.qprogram import remove_barrier

定义必要的函数

In [ ]:
def calculator(qp: QEnv = None, qc: BackendName = None) -> float:
    # Set the backend
    qp.backend(qc)

    # If the backend is BackendName.CloudBaiduQPUQian,
    # we disable the auto mapping module and remove all possible barrier gates.
    # Currently, barrier gates are not supported in BackendName.CloudBaiduQPUQian.
    if qc == BackendName.CloudBaiduQPUQian:
        qp.serverModule(ServerModule.Mapping, {"disable": True})
        remove_barrier(qp)

    # Commit the computation task and fetch the results
    res = qp.commit(shots=1024)
    # Set the quantum observable
    A = np.diag([1, 0]).astype(complex)

    try:
        with open(res['origin'], 'r') as f:
            data_origin = json.load(f)
            # use 'countsCorrected' when the backend is an actual Quantum Computer
            # TODO: We can always use the "counts" data when we have the "measurement error mitigation" module
            counts_name = "countsCorrected" if "countsCorrected" in data_origin.keys() else "counts"
            counts = data_origin[counts_name]
    except EnvironmentError:  # parent of IOError, OSError *and* WindowsError where available
        print("Opening file {} error!")

    print('---------')
    print('counts:', counts)
    return utils.expval_from_counts(A, counts)


def inv_opr(ops: List[QOperation]) -> RotationGate.RotationGateOP:
    """
    Computer inverse operation of a series preceding quantum operations.

    :param ops: list including a series of `QOperation`-type elements
    :return: a `QOperation`-type inverse quantum operation
    """
    inv_mat = functools.reduce(np.dot, [g.getInverse().getMatrix() for g in ops])
    # yzy-decomposition of a U3 gate
    _, theta, phi, lam = utils.decompose_yzy(inv_mat)
    # construct a U3 gate
    return RotationGate.U(theta, phi, lam)

为使用 PACS 的 10-qubit 超导量子计算机，用户需要从[量易伏](http://quantum-hub.baidu.com)中获取 token。

In [1]:
# 请登录“量易伏”平台（https://quantum-hub.baidu.com/）获取 Token
Define.hubToken = "TOKEN"

In [ ]:
# Set the necessary parameters
num_seq = 10
seed = 123

# folding 系数
scale_factors = [1, 2]

# 依次迭代 num_seq 次
val_ideal = []  # (num_seq,)
# val_miti_linear = []
val_miti_richard = []
# val_noise_linear = []  # (num_seq, num_scales)
val_noise_richard = []  # (num_seq, num_scales)

# ZNE mitigator
mtgr_richard = zne.ZNEMitigator(folder=zne.CircuitFolder(method='right'), extrapolator='richardson')

for i in range(1, num_seq + 1):
    qp = QEnv()
    qp.Q.createList(1)
    # Randomly generate and operate a list of Clifford gates
    cliff_seq = clifford.single_random_clifford(i, seed)
    for g in cliff_seq:
        g(qp.Q[0])

    print('>>>>>>>>>>>>>>>> Clifford length:', len(cliff_seq))

    # Compute and operate the inverse gate, making the circuit an identity-equivalent circuit
    inv_g = inv_opr(cliff_seq)
    inv_g(qp.Q[0])
    MeasureZ(*qp.Q.toListPair())

    # ideal value
    val = calculator(copy.deepcopy(qp), BackendName.LocalBaiduSim2)
    val_ideal.append(val)
    
    # noisy values and error-mitigated value
    val = mtgr_richard.mitigate(qp, BackendName.CloudBaiduQPUQian, calculator, scale_factors=scale_factors)
    val_miti_richard.append(val)
    val_noise_richard.append(mtgr_richard.history['expectations'])

# Richardson EM result
val_noise_richard = np.array(val_noise_richard).transpose()  # (num_scales, num_seq)
fig = visualization.plot_zne_sequences(val_ideal, val_miti_richard, val_noise_richard,
                                       scale_factors=scale_factors, title='richadson-CloudBaiduQPUQian-little')
fig.show()

**双比特 Clifford 随机测试**

In [ ]:
def calculator(qp: QEnv = None, qc: BackendName = None) -> float:
    """
    To set.

    :param qp: instance of `QEnv`, describes the quantum program
    :param qc: instance of `backend`, specifies the quantum computer
    :return: the evaluated expectation value
    """
    # Set the backend
    qp.backend(qc)

    # If the backend is BackendName.CloudBaiduQPUQian,
    # we disable the auto mapping module and remove all possible barrier gates.
    # Currently, barrier gates are not supported in BackendName.CloudBaiduQPUQian.
    if qc == BackendName.CloudBaiduQPUQian:
        qp.serverModule(ServerModule.Mapping, {"disable": True})
        remove_barrier(qp)

    # Commit the computation task and fetch the results
    res = qp.commit(shots=1024)
    # Set the quantum observable
    A = np.diag([1, -1, -1, 1]).astype(complex)

    try:
        with open(res['origin'], 'r') as f:
            data_origin = json.load(f)
            # use 'countsCorrected' when the backend is an actual Quantum Computer
            # TODO: We can always use the "counts" data when we have the "measurement error mitigation" module
            counts_name = "countsCorrected" if "countsCorrected" in data_origin.keys() else "counts"
            counts = data_origin[counts_name]
    except EnvironmentError:  # parent of IOError, OSError *and* WindowsError where available
        print("Opening file {} error!")

    print('---------')
    print('counts:', counts)
    return utils.expval_from_counts(A, counts)

In [ ]:
# Set the necessary parameters
num_seq = 3
seed = 123

# folding 系数
scale_factors = [1, 2]

# 依次迭代 num_seq 次
val_ideal = []  # (num_seq,)
# val_miti_linear = []
val_miti_richard = []
# val_noise_linear = []  # (num_seq, num_scales)
val_noise_richard = []  # (num_seq, num_scales)

# ZNE mitigator
mtgr_richard = zne.ZNEMitigator(folder=zne.CircuitFolder(method='right'), extrapolator='richardson')

for i in range(1, num_seq + 1):
    qp = QEnv()
    qp.Q.createList(2)
    # initial operations: Hadamard + CNOT
    FixedGate.H(qp.Q[0])
    FixedGate.CX(qp.Q[0], qp.Q[1])

    # Randomly generate and operate a list of Clifford gates
    cliff_seq = clifford.two_random_clifford(i, seed)  # type: List[List[List]]
    cliff_layers = sum(cliff_seq, [])  # type:List[List]

    # normal sequence
    for layer in cliff_layers:
        if len(layer) == 2:
            layer[0](qp.Q[0])
            layer[1](qp.Q[1])
        else:
            layer[0](qp.Q[0], qp.Q[1])

    # reserve sequence, making the circuit an identity-equivalent circuit
    for layer in reversed(cliff_layers):
        if len(layer) == 2:
            layer[0].getInverse()(qp.Q[0])
            layer[1].getInverse()(qp.Q[1])
        else:
            layer[0].getInverse()(qp.Q[0], qp.Q[1])

    print('>>>>>>>>>>>>>>>> Clifford length:', len(cliff_seq))

    MeasureZ(*qp.Q.toListPair())

    ##############################
    # 理想值
    ##############################
    val = calculator(copy.deepcopy(qp), BackendName.LocalBaiduSim2)
    val_ideal.append(val)

    ##############################
    # 含噪声情况和噪声缓释后的结果
    ##############################
    val = mtgr_richard.mitigate(qp, BackendName.CloudBaiduQPUQian, calculator, scale_factors=scale_factors)
    val_miti_richard.append(val)
    val_noise_richard.append(mtgr_richard.history['expectations'])
    
# Richardson EM result
val_noise_richard = np.array(val_noise_richard).transpose()  # (num_scales, num_seq)
fig = visualization.plot_zne_sequences(val_ideal, val_miti_richard, val_noise_richard,
                                       scale_factors=scale_factors, title='richadson-two-qubit-CloudBaiduQPUQian')
fig.show()

## 总结

酉折叠（Unitary Folding）方法的优势是在量子电路层面实现零噪声外插方法，抽象层度高且便于分析和理解。此外，在实现上更为简单，因为只需要不断的“拷贝拼接”量子电路或者量子门即可，而无需在量子控制层面考虑噪声的缩放情况。由实验结果来看，酉折叠可以结合常用的外插方法给出比较好的提升效果。

## 参考资料

\[1\] He, Andre, et al. "Zero-noise extrapolation for quantum-gate error mitigation with identity insertions." [Physical Review A](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.102.012426) 102.1 (2020): 012426.

\[2\] Giurgica-Tiron, Tudor, et al. "Digital zero noise extrapolation for quantum error mitigation." [2020 IEEE International Conference on Quantum Computing and Engineering (QCE)](https://ieeexplore.ieee.org/abstract/document/9259940/). IEEE, 2020.

\[3\] Kandala, Abhinav, et al. "Error mitigation extends the computational reach of a noisy quantum processor." [Nature](https://www.nature.com/articles/s41586-019-1040-7) 567.7749 (2019): 491-495.